In [92]:
def used_in_row(board, row, num):
    return num in board[row]

def used_in_col(board, col, num):
    return any(board[row][col] == num for row in range(9))

def used_in_box(board, box_row, box_col, num):
    for row in range(box_row, box_row + 3):
        for col in range(box_col, box_col + 3):
            if board[row][col] == num:
                return True
    return False

def is_valid_move(board, row, col, num):
    box_row = 3 * (row // 3)
    box_col = 3 * (col // 3)
    return not used_in_row(board, row, num) and not used_in_col(board, col, num) and not used_in_box(board, box_row, box_col, num)

def solve_sudoku_backtracking(board, row=0, col=0):
    # Puzzle solved
    if row == 9:
        return True

    # End of row
    if col == 9:
        return solve_sudoku_backtracking(board, row + 1, 0)

    # Cell already filled (by input)
    if board[row][col] != 0:
        return solve_sudoku_backtracking(board, row, col + 1)

    for num in range(1, 10):
        if is_valid_move(board, row, col, num):
            board[row][col] = num
            if solve_sudoku_backtracking(board, row, col + 1):
                return True
            board[row][col] = 0

    return False

In [93]:
from constraint import Problem, AllDifferentConstraint
import time

def print_board(board):
    for row in board:
        print(" ".join(str(val) if val != 0 else "_" for val in row))

def solve_sudoku_CSP(grid):
    # Variables X
    X = [(row, col) for row in range(9) for col in range(9)]
    
    # Domains D per variable - merging D from the lecture with the constraint that we must keep the cells in the input
    D = {}
    for (row, col) in X:
        val = grid[row][col]
        D[(row, col)] = [val] if val != 0 else list(range(1, 10))
    
    # Constraints C
    C = []
    
    # Rows constraint
    for row in range(9):
        C.append((AllDifferentConstraint(), [(row, col) for col in range(9)]))
    
    # Columns constraint
    for col in range(9):
        C.append((AllDifferentConstraint(), [(row, col) for row in range(9)]))
    
    # Block constraint
    for block_row in [0, 3, 6]:
        for block_col in [0, 3, 6]:
            block = [
                (r, c)
                for r in range(block_row, block_row + 3)
                for c in range(block_col, block_col + 3)
            ]
            C.append((AllDifferentConstraint(), block))
    
    problem = Problem()

    for var, dom in D.items():
        problem.addVariable(var, dom)
    
    for cons, variables in C:
        problem.addConstraint(cons, variables)
    
    sol = problem.getSolution()
    if not sol:
        return None
    
    # Return board
    return [[sol[(row, col)] for col in range(9)] for row in range(9)]

original_puzzle = [
[0, 0, 0, 0, 7, 0, 0, 0, 0],
[6, 0, 0, 1, 9, 5, 0, 0, 0],
[0, 9, 8, 0, 0, 0, 0, 6, 0],
[8, 0, 0, 0, 6, 0, 0, 0, 3],
[4, 0, 0, 8, 0, 3, 0, 0, 1],
[7, 0, 0, 0, 2, 0, 0, 0, 6],
[0, 6, 0, 0, 0, 0, 2, 8, 0],
[0, 0, 0, 4, 1, 9, 0, 0, 5],
[0, 0, 0, 0, 8, 0, 0, 7, 9]]


#puzzle = [[0]*9 for i in range(9)]

num_attempts = 1
csp_times, backtracking_times = [], []

# CSP
print("Puzzle:")
print_board(original_puzzle)

for _ in range(num_attempts):
    start_time = time.perf_counter()
    solution = solve_sudoku_CSP(original_puzzle)
    run_time = time.perf_counter() - start_time
    csp_times.append(run_time)

if solution:
    print("\nCSP Solution:")
    print_board(solution)
    print(f"\nAverage CSP Time taken (number of attempts = {num_attempts}) - {(sum(csp_times) / num_attempts):.4f}")


# Backtracking
import copy
for _ in range(num_attempts):
    puzzle = copy.deepcopy(original_puzzle)
    start_time = time.perf_counter()
    solution = solve_sudoku_backtracking(puzzle)
    run_time = time.perf_counter() - start_time
    backtracking_times.append(run_time)

if solution:
    print("\nBacktracking Solution:")
    print_board(puzzle)
    print(f"\nBacktracking Time taken (number of attempts = {num_attempts}) - {(sum(backtracking_times) / num_attempts):.4f}")
else:
    print("\nNo solution found.")

Puzzle:
_ _ _ _ 7 _ _ _ _
6 _ _ 1 9 5 _ _ _
_ 9 8 _ _ _ _ 6 _
8 _ _ _ 6 _ _ _ 3
4 _ _ 8 _ 3 _ _ 1
7 _ _ _ 2 _ _ _ 6
_ 6 _ _ _ _ 2 8 _
_ _ _ 4 1 9 _ _ 5
_ _ _ _ 8 _ _ 7 9

CSP Solution:
5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9

Average CSP Time taken (number of attempts = 1) - 0.0030

Backtracking Solution:
3 4 5 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
5 3 4 2 8 6 1 7 9

Backtracking Time taken (number of attempts = 1) - 0.1335
